# Qiskit Multirunner

### Imports

In [3]:
import json
import csv

import qiskit
from qiskit import QuantumCircuit, QuantumRegister
import qiskit_aer
from mqt import ddsim
from qiskit_ibm_runtime import QiskitRuntimeService

from qiskit_aer import AerSimulator
from qiskit_qulacs import QulacsProvider

### Sample Circuit

In [4]:
def get_sample_circuit() -> QuantumCircuit:
    qubits = QuantumRegister(1)
    qc = QuantumCircuit(qubits)
    qc.h(qubits[0])
    qc.measure_all()
    return qc

In [5]:
# Display our sample circuit
qc = get_sample_circuit()
qc.draw()

┌───┐ ░ ┌─┐
    q1: ┤ H ├─░─┤M├
        └───┘ ░ └╥┘
meas: 1/═════════╩═
                 0

### Qiskit AER Execution

In [6]:
def run_on_aer(qc: QuantumCircuit):
    backend = AerSimulator()
    result = backend.run(qc).result()
    return result
run_on_aer(get_sample_circuit())

Result(backend_name='aer_simulator', backend_version='0.16.3', qobj_id='', job_id='7fd0398a-2543-4a62-8012-aa9a588331d0', success=True, results=[ExperimentResult(shots=1024, success=True, meas_level=2, data=ExperimentResultData(counts={'0x1': 495, '0x0': 529}), header=QobjExperimentHeader(creg_sizes=[['meas', 1]], global_phase=0.0, memory_slots=1, n_qubits=1, name='circuit-163', qreg_sizes=[['q2', 1]], metadata={}), status=DONE, seed_simulator=2459301617, metadata={'num_bind_params': 1, 'runtime_parameter_bind': False, 'parallel_state_update': 4, 'parallel_shots': 1, 'sample_measure_time': 0.001278519, 'noise': 'ideal', 'batched_shots_optimization': False, 'remapped_qubits': False, 'active_input_qubits': [0], 'device': 'CPU', 'time_taken': 0.006751754, 'measure_sampling': True, 'num_clbits': 1, 'max_memory_mb': 8192, 'input_qubit_map': [[0, 0]], 'num_qubits': 1, 'method': 'stabilizer', 'required_memory_mb': 0, 'fusion': {'enabled': False}}, time_taken=0.006751754)], date=2025-03-27T22:

### Qiskit MQT Execution

In [7]:
def run_on_mqt(qc: QuantumCircuit):
    backend = ddsim.DDSIMProvider().get_backend("qasm_simulator")
    result = backend.run(qc).result()
    return result

run_on_mqt(get_sample_circuit())

Result(backend_name='qasm_simulator', backend_version='1.24.0', qobj_id='None', job_id='5b0738d5-316c-40e2-a1d7-5237eb9c52e2', success=True, results=[ExperimentResult(shots=1024, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 517, '0x1': 507}, time_taken=0.020756006240844727), header=DDSIMHeader(name='circuit-164', n_qubits=1, memory_slots=1, global_phase=0.0, creg_sizes=[('meas', 1)], clbit_labels=[('meas', 0)], qreg_sizes=[('q3', 1)], qubit_labels=[('q3', 0)]), status=DONE, seed=-1, metadata={})], date=None, status=COMPLETED, header=None, time_taken=0.02271580696105957)

### Qiskit Qulacs Execution

In [8]:
def run_on_qulacs(qc: QuantumCircuit):
    backend = QulacsProvider().get_backend('qulacs_simulator')
    result = backend.run(qc).result()
    return result

run_on_qulacs(get_sample_circuit())

/var/folders/0q/dj3wgc5576l38_bq_5tdyvf40000gn/T/ipykernel_1861/180786299.py:2: DeprecationWarning: The class ``qiskit.providers.provider.Provider`` is deprecated as of qiskit 1.1. It will be removed no earlier than 3 months after the release date. The abstract Provider and ProviderV1 classes are deprecated and will be removed in 2.0. You can just remove it as the parent class and a `get_backend` method that returns the backends from `self.backend`.
  backend = QulacsProvider().get_backend('qulacs_simulator')
/Users/mews/Documents/repos/qiskit-multirunner/.conda/lib/python3.11/site-packages/qiskit_qulacs/adapter.py:246: UserWarning: qiskit_qulacs.adapter: The Barrier instruction is not supported by Qiskit-Qulacs and has not been added to the circuit.
  warnings.warn(
/Users/mews/Documents/repos/qiskit-multirunner/.conda/lib/python3.11/site-packages/qiskit_qulacs/adapter.py:246: UserWarning: qiskit_qulacs.adapter: The Measure instruction is not supported by Qiskit-Qulacs and has not bee

Result(backend_name='qulacs_simulator', backend_version='0.1.0', qobj_id='0', job_id='a07c0e8c-d6ee-4e7b-80e3-dd9feec49f7b', success=True, results=[ExperimentResult(shots=0, success=True, meas_level=2, data=ExperimentResultData(statevector=[0.70710678+0.j 0.70710678+0.j]), status=JobStatus.DONE)], date=None, status=JobStatus.DONE, header=None, time_taken=0.0008740425109863281)

In [20]:
def log_1000_simulations_per_sumulator():
    samples = []
    circuit = get_sample_circuit()
    for i in range(0,999):
        execution_dict = run_on_aer(circuit).to_dict()
        samples.append(execution_dict)
    for i in range(0,999):
        execution_dict = run_on_mqt(circuit).to_dict()
        samples.append(execution_dict)
    for i in range(0,999):
        execution_dict = run_on_qulacs(circuit).to_dict()
        samples.append(execution_dict)
    return samples

def executions_to_csv(samples: list, target_file: str):
    file = open(target_file, 'r+')
    #get a sample of the dataset for write parameters
    sample = samples[0]
    #write the type header
    header = ','.join(sample.keys())
    file.write(header)

    writer = csv.DictWriter(file, fieldnames=list(sample.keys()))

    for i in samples:
        writer.writerow(i)

    file.close()

    pass

execs = log_1000_simulations_per_sumulator()
executions_to_csv(execs, '../output/simulations.csv')

/var/folders/0q/dj3wgc5576l38_bq_5tdyvf40000gn/T/ipykernel_1861/180786299.py:2: DeprecationWarning: The class ``qiskit.providers.provider.Provider`` is deprecated as of qiskit 1.1. It will be removed no earlier than 3 months after the release date. The abstract Provider and ProviderV1 classes are deprecated and will be removed in 2.0. You can just remove it as the parent class and a `get_backend` method that returns the backends from `self.backend`.
  backend = QulacsProvider().get_backend('qulacs_simulator')
/Users/mews/Documents/repos/qiskit-multirunner/.conda/lib/python3.11/site-packages/qiskit_qulacs/adapter.py:246: UserWarning: qiskit_qulacs.adapter: The Barrier instruction is not supported by Qiskit-Qulacs and has not been added to the circuit.
  warnings.warn(
/Users/mews/Documents/repos/qiskit-multirunner/.conda/lib/python3.11/site-packages/qiskit_qulacs/adapter.py:246: UserWarning: qiskit_qulacs.adapter: The Measure instruction is not supported by Qiskit-Qulacs and has not bee

# Real-hardware execution

Configure the following with your account in order to execute jobs in actual IBM hardware, it implies a compute time cost, so you should probably be cautious about running this multiple times.

In [ ]:
def run_on_hw(qc: QuantumCircuit):
    backend = QiskitRuntimeService().backend('ibm_brisbane')
    result = backend.run(qc).result()
    return result

run_on_hw(get_sample_circuit())

AccountNotFoundError: 'Unable to find account.'